In [1]:
import random
import time

## Battleship Outline

1. Includes a 8 x 8 grid and 6 ships with varying lengths (between length 2 and 5 random).
2. Will have Easy, Medium, Hard, and Expert difficulties
    -- These difficulties are defined by the number of shots you have to sink all ships
    -- Works the same way as a computer opponent that wins in X turns.
    Easy -> 58 shots,
    Medium -> 48 shots,
    Hard  -> 38 shots,
    Expert -> 28 shots,
3. Rest of the game is standard, will be able to choose squares on the grid and the grid will display hits and misses
4. Ships cannot run diagonal.
5. You lose if you run out of shots without sinking all ships

## Legend

1.  " S " -> Part of a Ship
2.  " U " -> Unkown Water
3.  " X " -> Hit a ship
4.  " # " -> Shot fired and missed

In [2]:
## Initialization

grid = [[]]     #Create grid
gridSize = 8   #Initialize length/width of grid
shipPosition = [[]]  #Create grid for ship position
numbShips = 6    #Initialize total number of ships
numbSunk = 0    #Game starts with no ships sunk
gameEnded = False   #Create end 

#Create various game modes defined by how many shots you have to sink
#Easy == 1, Med == 2, Hard == 3, Expert == 4
modeEasy = 58
modeMedium = 48
modeHard = 38
modeExpert = 28
shotV = [modeEasy, modeMedium, modeHard, modeExpert]
alpha = "ABCDEFGHIJKL"
shotsLeft = []

#Game Functions

def chooseDifficulty():
    global shotsLeft
    while not shotsLeft:
        difficulty = input("Enter value between 0 -> Easy and 3 -> Expert")
        difficulty = int(difficulty)
        if 0 <= difficulty < 4:
            shotsLeft = shotV[difficulty]
            print(shotsLeft)
            return shotsLeft
        else:
            print("Enter a difficult between 0 -> Easy and 3 -> Expert")
            continue
    

def valShipPlace(rowStart, rowEnd, colStart, colEnd):
    #Ensures ship can be place (valid location)

    global grid
    global shipPosition
    
    validBoard = True
    for r in range(rowStart, rowEnd):
        for c in range(colStart, colEnd):
            if grid[r][c] != "U":
                validBoard = False
                break
    
    if validBoard:
        shipPosition.append([rowStart, rowEnd, colStart, colEnd])
        for r in range(rowStart, rowEnd):
            for c in range(colStart, colEnd):
                grid[r][c] = "S"
    return validBoard

def actionPlaceShip(row, col, direction, len):
    #Places the ship once the location is valid
    
    global gridSize
    #initialize starting point for the ship
    rowStart, rowEnd, colStart, colEnd = row, row + 1, col, col + 1
    if direction == "left":
        if col - len < 0:
            return False
        colStart = col - len + 1  #account for offset
        
    elif direction == "right":
        if col + len >= gridSize:
            return False
        colEnd = col + len
        
    elif direction == "up":
        if row - len < 0: 
            return False
        rowStart = row - len + 1 #account for offset
        
    elif direction == "down":
        if row + len >= gridSize:
            return False 
        rowEnd = row + len
    
    return valShipPlace(rowStart, rowEnd, colStart, colEnd)

def createGrid():
    #Creates grid for the game and sets difficulty
    #Sets difficulty -> Easy == 1, Med == 2, Hard == 3, Expert == 4
    
    global grid
    global gridSize
    global numbShips
    global shipPosition 
    
    random.seed(time.time())
    
    rows, cols = (gridSize, gridSize)
    
    grid = []
    for r in range(rows):
        row = []
        for c in range(cols):
            row.append("U")
        grid.append(row)
    
    numbShipsPlaced = 0
    
    shipPosition = []
    
    while numbShipsPlaced != numbShips:
        randomRow = random.randint(0, rows - 1)
        randomCol = random.randint(0, cols - 1)
        direction = random.choice(["left", "right", "up", "down"])
        shipSize = random.randint(2, 5)
        if actionPlaceShip(randomRow, randomCol, direction, shipSize):
            numbShipsPlaced += 1
    
def showGrid():
    #Displays grid with appropiate mapping
    
    global grid
    global alpha
    
    debugMode = True   #This is to test for scripting purposes if the game is functioning, playing the game will turn this False
    
    alpha = alpha[0: len(grid) + 1]
    
    for row in range(len(grid)):
        print(alpha[row], end =") ")
        for col in range(len(grid[row])):
            if grid[row][col] == "S":
                if debugMode:
                    print("S", end = " ")
                else:
                    print("U", end =" ")
            else:
                print(grid[row][col], end = " ")
        print("")
    
    print("  ", end=" ")
    for i in range(len(grid[0])):
        print(str(i), end=" ")
    print("")

def valShot():
    #Will ensure valid shot placement
    
    global grid
    global alpha
    
    validPlacement = False
    row = -1
    col = -1
    while validPlacement is False:
        placement = input("Enter row {A - H} and column {0 - 7} such as B2: ")
        placement = placement.upper()
        if len(placement) <= 0 or len(placement) > 2: #checking if too few or to many characters were inputted
            print("Invalid Location: Please enter single row and single column")
            continue
        row = placement[0]
        col = placement[1]
        if not row.isalpha() or not col.isnumeric(): #checks for valid input letter and number
            print("Invalid Entry: Please enter letter {A - H} for row and {0 - 7} for column")
            continue
        row = alpha.find(row)
        if not (-1 < row < gridSize): #Makes sure the entry is inside the grid
            print("Invalid Entry: Please enter letter {A - H} for row and {0 - 7} for column")
            continue
        col = int(col)
        if not (-1 < col < gridSize): #Makes sure the entry is inside the grid
            print("Invalid Entry: Please enter letter {A - H} for row and {0 - 7} for column")
            continue
        if grid[row][col] == "#" or grid[row][col] == "X":
            print("A bullet has already been shot here, choose another location")
            continue
        if grid[row][col] == "U" or grid[row][col] == "S":
            validPlacement =True
            
    return row, col

def checkSunk(row, col):
    #Boolean return for whether or not a ship has been sunk
    global shipPosition
    global grid
    
    for position in shipPosition:
        startRow = position[0]
        endRow = position[1]
        startCol = position[2]
        endCol = position[3]
        if startRow <= row <= endRow and startCol <= col <= endCol:
            #Ship is found, now we check is it is sunk
            for r in range(startRow, endRow):
                for c in range(startCol, endCol):
                    if grid[r][c] != "X":
                        return False
    return True
    
    
def fireShot():
    #Changes grid based on location of shot
    
    global grid
    global numbSunk
    global shotsLeft
    
    row, col = valShot()
    print("")
    print("---------------------------")
    
    if grid[row][col] == "U":
        print("Shot Missed")
        grid[row][col] = "#"
    elif grid[row][col] == "S":
        print("Shot Hit!", end= " ")
        grid[row][col] = "X"
        if checkSunk(row, col):
            print("Ship has been sunk.")
            numbSunk += 1
        else:
            print("Ship has been hit")
            
    shotsLeft -= 1
    
def checkGameOver():
    
    global numbSunk
    global numbShips
    global shotsLeft
    global gameEnded
    
    if numbShips == numbSunk:
        print("You have won!")
        gameEnded = True
    elif shotsLeft <= 0:
        print("No more shots remaining, you have lost. Good luck next time!")
        gameEnded = True

##Game Loop

def main():
    global gameEnded
    
    print("----Welcome to Battleship----")
    print("Choose difficulty")
    
    chooseDifficulty()
    createGrid()
    
    while gameEnded is False:
        showGrid()
        print("Number of ships remaining: " + str(numbShips - numbSunk))
        print("Number of shots left: " + str(shotsLeft))
        fireShot()
        print("-------------------------")
        print("")
        checkGameOver()


if __name__ == "__main__":
    main()

----Welcome to Battleship----
Choose difficulty
58
A) U U U U U U U U 
B) U U U U U U U U 
C) U U U U S U U U 
D) S S S U S U U U 
E) U U U U S S U U 
F) U U U U S S S U 
G) U U U U S S S U 
H) S S S S U S S U 
   0 1 2 3 4 5 6 7 
Number of ships remaining: 6
Number of shots left: 58

---------------------------
Shot Missed
-------------------------

A) # U U U U U U U 
B) U U U U U U U U 
C) U U U U S U U U 
D) S S S U S U U U 
E) U U U U S S U U 
F) U U U U S S S U 
G) U U U U S S S U 
H) S S S S U S S U 
   0 1 2 3 4 5 6 7 
Number of ships remaining: 6
Number of shots left: 57

---------------------------
Shot Hit! Ship has been hit
-------------------------

A) # U U U U U U U 
B) U U U U U U U U 
C) U U U U S U U U 
D) X S S U S U U U 
E) U U U U S S U U 
F) U U U U S S S U 
G) U U U U S S S U 
H) S S S S U S S U 
   0 1 2 3 4 5 6 7 
Number of ships remaining: 6
Number of shots left: 56

---------------------------
Shot Hit! Ship has been hit
-------------------------

A) # U U U 

## Future Applications

1. Can include smarter computer that uses a priority algorithm that functions more human-like. This would be done by choosing every other square optimizing the fewest number of shots and then prioritizing neighboring squares if a hit is found.
2. Can include a second player so you can play against a friend.